<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.1: Generators: Parameters | 生成器: 变量
**Prev:上一个小节： [Putting it all Together: An FIR Filter | FIR 滤波器](2.5_fir.ipynb)**<br>
**Next:下一个小节： [Generators: Collections | 生成器: 集合](3.2_collections.ipynb)**

## Motivation | 动机
For Chisel modules to be code generators, there must be something that tells the generator how it should go about its job. 
In this section we discuss module parameterization, the various methodologies and Scala language features. 
The richness of the parameter passing implementation is directly proportional to the richness of the circuits generated. 
Parameters should provide useful default values, be easy to set, and protect against illegal or non-sensical values. 
For more complicated system it is very useful if they can be locally overriden in a way that does not inadvertantly affect other modules usages. 

为了完成 Chisel 模块的生成器，代码中必须要有一些告知生成器如何工作的部分。
在这个小节中我们会讨论模块参数化，这是一个多方面的方法学也是 Scala 语言的特性。
在这个小节中，我们会讨论一个灵活运用的方法，也是 Scala 语言的特征——模块参数化。
变量应该提供一些有用的默认参数值，使得变量容易设置、防止非法或者无意义的参数值。
如果这些参数可以在模块内覆写并且不会影响到其他模块的使用，那么模块参数化将会在复杂的系统当中非常有用。

## | 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# Parameter Passing | 参数传递
Chisel provides powerful constructs for writing hardware generators. 
Generators are programs that take some circuit parameters and produce a circuit description. 
In this section, we'll start with discussing how Chisel generators get their parameters. 

Chisel 给硬件生成器提供了非常有用的组成。
生成器是一些接收电路参数然后生成电路描述的程序片段。
在这个小节中，我们将讨论 Chisel 生成器如何接收参数。

<span style="color:blue">**Example: Parameterized Scala Object | 举例：参数化 Scala 对象**</span><br>
Every Chisel `Module`s is a Scala class just like any other. 
Recall that Scala classes can be parameterized like so: 

每一个 Chisel 的`Module`只是一个普通的 Scala 类
回想一下，Scala 中的类可以这样变成参数化的：

In [ ]:
class ParameterizedScalaObject(param1: Int, param2: String) {
  println(s"I have parameters: param1 = $param1 and param2 = $param2")
}
val obj1 = new ParameterizedScalaObject(4,     "Hello")
val obj2 = new ParameterizedScalaObject(4 + 2, "World")

<span style="color:blue">**Example: Parameterized Chisel Object | 举例：参数化 Chisel 对象**</span><br>
Chisel modules can be parameterized the same way.
The following module has parameters for the widths of all its inputs and outputs. 
Running the code block will print generated Verilog.
Play with the parameters and check that the output changes to reflect new parameters. 

Chisel 的模块也可以像这样变成参数化的。
下面这个模块所有的输入、输出的位宽都是参数化的。
运行下面的代码块会打印生成的 Verilog。
你可以尝试改变参数，然后看看输出是怎么随着参数改变的。


In [ ]:
class ParameterizedWidthAdder(in0Width: Int, in1Width: Int, sumWidth: Int) extends Module {
  require(in0Width >= 0)
  require(in1Width >= 0)
  require(sumWidth >= 0)
  val io = IO(new Bundle {
    val in0 = Input(UInt(in0Width.W))
    val in1 = Input(UInt(in1Width.W))
    val sum = Output(UInt(sumWidth.W))
  })
  // a +& b includes the carry, a + b does not
  io.sum := io.in0 +& io.in1
}

println(getVerilog(new ParameterizedWidthAdder(1, 4, 6)))

The above code block has some `require(...)` statements. 
These are pre-elaboration assertions, which are useful when your generator only works with certain parameterizations or when some parameterizations are mutually exclusive or nensensical. 
The above code block checks that widths are non-negative. 

There is a separate construct for simulation-time assertions called `assert(...)`. 

上面的代码块有一些`require(...)`声明。
这些是在实现前的断言，当你的生成器只在某些特定参数运行或者一些参数是相互排斥或者无意义的时候，这就会变的很有用。
上方代码块会检查位宽是否是非负数。

另外还有一个为了仿真时间断言的独立结构——`assert(...)`


## Sorting with Parameterized Modules | 使用参数化模块实现排序功能
The following code block is a parameterized sort similar to `Sort4` from module 2.3. 
Unlike the previous example of an adder with parameterized width IOs, this example has a fixed IO. 
The parameter controls what hardware is generated inside the module. 

下方的代码块是一个与模块2.3中的`Sort4`相似的参数化排序功能。
之前的例子中，加法器有一个参数化位宽的IO，但在这个例子中IO的位宽是固定的。
这个参数控制的是在模块内会生成什么样的硬件。
![Sort4](images/Sorter4.png)
<span style="color:blue">**Example: Parameterized 4-Input Sort | 举例：参数化的四输入排序电路**</span><br>
Unlike 2.3, this implementation is parameterized to be either a descending or an ascending sort. 

不像2.3中的电路，这次的实现方法是使用参数，实现递减排序或者递增排序。

In [ ]:
/** Sort4 sorts its 4 inputs to its 4 outputs *//** Sort4 把它的四个输入排序后输出 */
class Sort4(ascending: Boolean) extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })
    
  // this comparison funtion decides < or > based on the module's parameterization 
  //这个比大小的功能是基于模块的参数来判断是应该取大于号还是小于号。
  def comp(l: UInt, r: UInt): Bool = {
      if (ascending) {
        l < r
      } else {
        l > r
    }
  }

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(comp(io.in0, io.in1)) {
    row10 := io.in0            // preserve first two elements 前两个元素保持不变
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // swap first two elements 交换前两个元素
    row11 := io.in0
  }

  when(comp(io.in2, io.in3)) {
    row12 := io.in2            // preserve last two elements 后两个元素保持不变
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // swap last two elements 交换后两个元素
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(comp(row11, row12)) {
    row21 := row11            // preserve middle 2 elements 保留中间两个元素
    row22 := row12
  }.otherwise {
    row21 := row12            // swap middle two elements 交换中间两个元素
    row22 := row11
  }

  val row31 = Wire(UInt(16.W))
  val row32 = Wire(UInt(16.W))
  when(comp(row10, row13)) {
    row31 := row10            // preserve middle 2 elements 保留中间两个元素
    row32 := row13
  }.otherwise {
    row31 := row13            // swap middle two elements 交换中间两个元素
    row32 := row10
  }

  when(comp(row10, row21)) {
    io.out0 := row31            // preserve first two elements 前两个元素保持不变
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements 交换前两个元素
    io.out1 := row31
  }

  when(comp(row22, row13)) {
    io.out2 := row22            // preserve first two elements 前两个元素保持不变
    io.out3 := row32
  }.otherwise {
    io.out2 := row32            // swap first two elements 交换前两个元素
    io.out3 := row22
  }
}

// verify the inputs are sorted 检查输入是否已经排好序了
class Sort4AscendingTester(c: Sort4) extends PeekPokeTester(c) {
  poke(c.io.in0, 3)
  poke(c.io.in1, 6)
  poke(c.io.in2, 9)
  poke(c.io.in3, 12)
  expect(c.io.out0, 3)
  expect(c.io.out1, 6)
  expect(c.io.out2, 9)
  expect(c.io.out3, 12)

  poke(c.io.in0, 13)
  poke(c.io.in1, 4)
  poke(c.io.in2, 6)
  poke(c.io.in3, 1)
  expect(c.io.out0, 1)
  expect(c.io.out1, 4)
  expect(c.io.out2, 6)
  expect(c.io.out3, 13)

  poke(c.io.in0, 13)
  poke(c.io.in1, 6)
  poke(c.io.in2, 4)
  poke(c.io.in3, 1)
  expect(c.io.out0, 1)
  expect(c.io.out1, 4)
  expect(c.io.out2, 6)
  expect(c.io.out3, 13)

}
class Sort4DescendingTester(c: Sort4) extends PeekPokeTester(c) {
  poke(c.io.in0, 3)
  poke(c.io.in1, 6)
  poke(c.io.in2, 9)
  poke(c.io.in3, 12)
  expect(c.io.out0, 12)
  expect(c.io.out1, 9)
  expect(c.io.out2, 6)
  expect(c.io.out3, 3)

  poke(c.io.in0, 13)
  poke(c.io.in1, 4)
  poke(c.io.in2, 6)
  poke(c.io.in3, 1)
  expect(c.io.out0, 13)
  expect(c.io.out1, 6)
  expect(c.io.out2, 4)
  expect(c.io.out3, 1)
    
  poke(c.io.in0, 1)
  poke(c.io.in1, 6)
  poke(c.io.in2, 4)
  poke(c.io.in3, 13)
  expect(c.io.out0, 13)
  expect(c.io.out1, 6)
  expect(c.io.out2, 4)
  expect(c.io.out3, 1)

}

// Here are the testers 以下是验证
val worksAscending = iotesters.Driver(() => new Sort4(true)) { c => new Sort4AscendingTester(c) }
val worksDescending = iotesters.Driver(() => new Sort4(false)) { c => new Sort4DescendingTester(c) }
assert(worksAscending && worksDescending) // Scala Code: if works == false, will throw an error Scala代码，如果 works == false，那么将会报错
println("成功！！") // Scala Code: if we get here, our tests passed! Scala 代码，如果我们运行到了这里，那么我们的验证就通过啦！

---
# Option and Default Arguments | 选项和默认的参数

There are times when a function sometimes returns a value, and sometimes does not. Instead of erroring when it cannot return a value, Scala has a mechanism to encode this in the type system. 

有时候一个函数会返回一个值，但有时候不会。当函数值不返回任何值的时候，Scala 有一个类型编码机制使得它不会报错。

<span style="color:blue">**Example: Erroneous Map Index Call | 举例：错误索引单元**</span><br>
In the following example, we have a map containing several key/value pairs. If we try to access a missing key/value pair, then we get a runtime error: 

在下面的例子中，我们有一个包含了一些键/值关系的映射。如果我们试着获取不存在的键/值关系，那么在运行时会报错：

In [ ]:
val map = Map("a" -> 1)
val a = map("a")
println(a)
val b = map("b")
println(b)

<span style="color:blue">**Example: Getting Uncertain Indices | 举例：获得不确定的索引**</span><br>
However, `Map` provides another way to access a key's value, through the **get** method. Using this returns a value of abstract class `Option`. `Option` has two subclasses, `Some` and `None`. 

然而，`Map`  也提供了另外一种获得键对应值的方法，那就是通过 **get**。使用 **get** 会返回一个抽象类 `Option` 的值，`Option` 下面又有两个小类，`Some` 和 `None`。

In [ ]:
val map = Map("a" -> 1)
val a = map.get("a")
println(a)
val b = map.get("b")
println(b)

As you'll see in later sections, `Option` is extremely important because it lets users use a match statement to check Scala types and values. 

正如你将在后面的小节中看到的，`Option` 非常重要因为它使得用户可以使用一个匹配语句来检查 Scala 的类型和值。

<span style="color:blue">**Example: Get Or Else! | 举例： Get 或是其他**</span><br>
Like `Map`, `Option` also has a `get` method, which errors if called on `None`. For these instances, we can provide a default using **`getOrElse`**. 

`Map` 和 `Option` 都有一个方法——`get`，如果得到的类型是 `None`，那么它将会报错。在这些情况下，我们可以提供一个默认的使用方法：**`getOrElse`**

In [ ]:
val some = Some(1)
val none = None
println(some.get)          // Returns 1 返回1
// println(none.get)       // Errors! 错误！
println(some.getOrElse(2)) // Returns 1 返回1
println(none.getOrElse(2)) // Returns 2 返回2

## Options for Parameters with Defaults | 带有默认值的选项

When objects or functions have a lot of parameters, it can be tedious and error-prone to fully specify them all the time. 
In module 1, you were introduced to named arguments and parameter defaults. 
Sometimes, a parameter doesn't have a good default value. 
`Option` can be used with a default value of `None` in these situations. 

当一个对象或者函数有很多参数时，如果我们需要同时给所有参数赋值，这将会是无聊的并且容易出错。
在模块1，你知道了对自变量和参数设定默认名字。
有时，一个参数没有一个好的默认值。
在这些情况下，`Option` 就可以被赋予一个 `None` 的默认值。

<span style="color:blue">**Example: Optional Reset | 举例：选择性重置**</span><br>
The following shows a block that delays its input by one clock cycle.
If `resetValue = None`, which is the default, the register will have no reset value and be initialized to garbage.
This avoids the common but ugly case of using values outside the normal range to indicate "none", like using -1 as the reset value to indicate that this register is not reset. 

下面将会展示一个输入延迟了一个时钟输出的模块。
如果默认值 `resetValue = None` 没有改变，那么寄存器没有复位值并且会被初始化为丢弃的寄存器。
这就避免了常见但是不好看的情况——需要使用在正常范围外的值来表明 `none`，就像使用 `-1` 作为复位值来表明这个寄存器不可以被复位的。 

In [ ]:
class DelayBy1(resetValue: Option[UInt] = None) extends Module {
    val io = IO(new Bundle {
        val in  = Input( UInt(16.W))
        val out = Output(UInt(16.W))
    })
    val reg = if (resetValue.isDefined) { // resetValue = Some(number)
        RegInit(resetValue.get)
    } else { //resetValue = None
        Reg(UInt())
    }
    reg := io.in
    io.out := reg
}

println(getVerilog(new DelayBy1))
println(getVerilog(new DelayBy1(Some(3.U))))

---
# Match/Case Statements | Match/Case 语句
The Scala *matching* concept is used throughout Chisel and needs to be part of any Chisel programmer's basic understanding. Scala provides the match operator which supports:
- Simple testing for alternatives, something like a C *switch* statement 
- More complex testing of ad-hoc combinations of values 
- Taking actions based on the type of a variable when its type is unknown or underspecified, for example when 
  - variable is taken from a heterogeneous list ```val mixedList = List(1, "string", false)``` 
  - or variable is known to be a member of a super-class but not which specific sub-class it is. 
- Extraction of sub strings of a string that are specified with a *regular expression* 

Scala *matching* 也在 Chisel 中使用，并且是任何一个 Chisel 程序的基本构成之一。Scala 提供了匹配操作，它支持以下内容：
- 为备选情况提供简单的测试，与 C 语言中的 *switch* 类似。
- 提供更加复杂的测试，特别是组合值。
- 当变量的类型是未知或者不是特定的，那么将根据变量的类型跳转做出不同的反应，比如说：
    - 当变量值从混杂列表中来时```val mixedList = List(1, "string", false)```
    - 或者当变量已知是某一种超类但是不知道具体属于哪一个子类时。
- 通过*正则表达式*从一个字符串中抽取一部分字符串。

<span style="color:blue">**Example: Value Matching | 举例：匹配值**</span><br>
The following example, depending on the **value** of the variable we **match** on, we execute a different **case** statement: 

下面这个例子中，我们使用不同的 **case** 语句来实现 **匹配** 变量的值。

In [ ]:
// y is an integer variable defined somewhere else in the code y 是一个在代码其他地方定义的整数变量。
val y = 7
/// ...
val x = y match {
  case 0 => "zero" // One common syntax, preferred if fits in one line 一种常见的语法，如果可以在一行内实现可以采用这种语法。
  case 1 =>        // Another common syntax, preferred if does not fit in one line. 另外一种常见语法，如果不能在一行内实现可以采用这种语法
      "one"        // Note the code block continues until the next case 注意代码块直到另外一个 case 才结束。
  case 2 => {      // Another syntax, but curly braces are not required 另外一种语法，但是尖括号不是必须的。
      "two"
  }
  case _ => "many" // _ is a wildcard that matches all values _ 是一个匹配所有值的通配符。
}
println("y is " + x)

The match operator checks possible values and for each case returns a string.  A couple of things to note: 
- Each code block that follows a ```=>``` operator continues until it reaches either the ending brace of the match or the next case statement. 
- A match is searched in the order of the case statements, once a case statement has been matched, no other checks against other case statements are made. 
- The use of underscore as a wildcard, to handle any value not found. 

匹配操作会检查所有可能的值并且每一个 `case` 都会返回一个字符串。有一些事情需要注意：
- 每一个紧跟着一个 ```=>``` 操作符的代码块都会继续直到右尖括号或者另外一个 `case` 语句。
- 匹配会顺着 case 语句的顺序搜索，一旦一个 `case` 语句被匹配到，那么不会有其他的 `case` 语句会被检查到。
- 下划线作为通配符来解决没有任何值被匹配。


<span style="color:blue">**Example: Multiple Value Matching | 举例：多个值匹配**</span><br>
Also, multiple variables can be matched at the same time. Here's a simple example of a truth table implemented with a match statement and tuple of values: 

并且，多个变量可以同时匹配。下面有一个使用匹配语句和值数组的方式实现真值表的简单例子。

In [ ]:
def animalType(biggerThanBreadBox: Boolean, meanAsCanBe: Boolean): String = {
  (biggerThanBreadBox, meanAsCanBe) match {
    case (true, true) => "wolverine"
    case (true, false) => "elephant"
    case (false, true) => "shrew"
    case (false, false) => "puppy"
  }
}
println(animalType(true, true))

<span style="color:blue">**Example: Type Matching | 举例：匹配类型**</span><br>
Scala is a strongly typed language, so the types of all objects are known during runtime. We can use **match statements** to use this type information to dictate control flow: 

Scala 是一个强类型的语言，所以所有对象在运行时都知道。我们可以使用**匹配语句**来利用类型信息指示控制流：

In [ ]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
  x match {
    case s: String => println(s"$x is a String")
    case s: Int    => println(s"$x is an Int")
    case s: Double => println(s"$x is a Double")
    case _ => println(s"$x is an unknown type!")
  }
}

<span style="color:blue">**Example: Multiple Type Matching | 举例：多个类型匹配**</span><br>
If you want to match on whether a value has one of many types, use the following syntax. *Note that you **must** use an `_` when matching.* 

如果你想匹配是否一个值有一个或多个类型，可以使用以下语法。*注意，你在匹配时**必须**使用一个 _ *

In [ ]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
  x match {
    case _: Int | _: Double => println(s"$x is a number!")
    case _ => println(s"$x is an unknown type!")
  }
}

<span style="color:blue">**Example: Type Matching and Erasure | 举例：类型匹配和擦除**</span><br>
Type matching has some limitations. Because Scala runs on the JVM, and the JVM does not maintain polymorphic types, you cannot match on them at runtime (because they are all erased). Note that the following example always matches the first case statement, because the `[String]`, `[Int]`, and `[Double]` polymorphic types are erased, and the case statements are **actually** matching on just a `Seq`. 

类型匹配也有一些限制。因为 Scala 运行在 JVM 中，但 JVM 不保留多形态的类型，在运行的时候你不能够匹配他们（因为他们都被擦除了）。注意在以下的例子中总会匹配到第一个 case 语句，因为`[String]`，`[Int]` 和 `[Double]` 这些多形态类型被擦除了，所以 case 语句**实际上**匹配的只是一个 `Seq`。

In [ ]:
val sequence = Seq(Seq("a"), Seq(1), Seq(0.0))
sequence.foreach { x =>
  x match {
    case s: Seq[String] => println(s"$x is a String")
    case s: Seq[Int]    => println(s"$x is an Int")
    case s: Seq[Double] => println(s"$x is a Double")
  }
}

Note that Scala compilers will usually give a warning if you implement code like the example above. 

注意：如果你像上述例子这样实现代码，那么 Scala 编译器实际上会报出警告。

<span style="color:blue">**Example: Optional Reset Matching | 举例：可选择地复位匹配**</span><br>
The following code block shows the same `DelayBy1` module with the match construct instead of `if/else`.

下述的代码块将会展示一个使用匹配语句而不是 `if/else` 实现的 `DelayBy1` 模块。

In [ ]:
class DelayBy1(resetValue: Option[UInt] = None) extends Module {
  val io = IO(new Bundle {
    val in  = Input( UInt(16.W))
    val out = Output(UInt(16.W))
  })
  val reg = resetValue match {
    case Some(r) => RegInit(r)
    case None    => Reg(UInt())
  }
  reg := io.in
  io.out := reg
}

println(getVerilog(new DelayBy1))
println(getVerilog(new DelayBy1(Some(3.U))))

---
# IOs with Optional Fields | 可选择性的启用 IO

Sometimes we want IOs to be optionally included or excluded. 
Maybe there's some internal state that's nice to be able to look at for debugging, but you want to hide it when the generator is being used in a system.
Maybe your generator has some inputs that don't need to be connected in every situation because there is a sensible default. 

有时候我们希望 IO 可以可选择性地包括或者排除。
可能在我们 debug 的时候，有一些中间态是很棒的，但是当这个生成器在一个系统当中使用的时候我们想要隐藏这些中间态。
可能你的生成器有一些不是在每种状态下都想要连接的输入端口，因为有一些明显的默认状态。

<span style="color:blue">**Example: Optional IO with Option | 举例：可选择的 IO**</span><br>
Optional bundle fields are one way to get this functionality. 
In the following example, we show a one-bit adder that optionally takes in a carry. 
If the carry is included, `io.carryIn` will have type `Some[UInt]` and be included in the IO bundle. 
If the carry is not included, `io.carryIn` will have type `None` and will be excluded from the IO bundle. 

可选择启用的 bundle 是实现这个功能的一种方法。
下面这个例子中，我们展示一个带有选择性进位的加法器。
如果需要进位，那么 `io.carryIn` 的类型是 `Some[UInt]`，并且在 IO bundle 中也要申明。
如果不需要进位，那么 `io.carryIn` 的类型是 `None` 并且也不需要在 IO bundle 中申明。

In [ ]:
class HalfFullAdder(val hasCarry: Boolean) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val carryIn = if (hasCarry) Some(Input(UInt(1.W))) else None
    val s = Output(UInt(1.W))
    val carryOut = Output(UInt(1.W))
  })
  val sum = io.a +& io.b +& io.carryIn.getOrElse(0.U)
  io.s := sum(0)
  io.carryOut := sum(1)
}

class HalfAdderTester(c: HalfFullAdder) extends PeekPokeTester(c) {
  require(!c.hasCarry, "DUT must be half adder    DUT必须是半加器")
  // 0 + 0 = 0
  poke(c.io.a, 0)
  poke(c.io.b, 0)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 0)
  // 0 + 1 = 1
  poke(c.io.b, 1)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)
  // 1 + 1 = 2
  poke(c.io.a, 1)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 1)
  // 1 + 0 = 1
  poke(c.io.b, 0)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)
}

class FullAdderTester(c: HalfFullAdder) extends PeekPokeTester(c) {
  require(c.hasCarry, "DUT must be half adder   DUT必须是半加器")
  poke(c.io.carryIn.get, 0)
  // 0 + 0 + 0 = 0
  poke(c.io.a, 0)
  poke(c.io.b, 0)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 0)
  // 0 + 0 + 1 = 1
  poke(c.io.b, 1)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)
  // 0 + 1 + 1 = 2
  poke(c.io.a, 1)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 1)
  // 0 + 1 + 0 = 1
  poke(c.io.b, 0)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)

  poke(c.io.carryIn.get, 1)
  // 1 + 0 + 0 = 1
  poke(c.io.a, 0)
  poke(c.io.b, 0)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)
  // 1 + 0 + 1 = 2
  poke(c.io.b, 1)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 1)
  // 1 + 1 + 1 = 3
  poke(c.io.a, 1)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 1)
  // 1 + 1 + 0 = 2
  poke(c.io.b, 0)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 1)
}

val worksHalf = iotesters.Driver(() => new HalfFullAdder(false)) { c => new HalfAdderTester(c) }
val worksFull = iotesters.Driver(() => new HalfFullAdder(true)) { c => new FullAdderTester(c) }
assert(worksHalf && worksFull) // Scala Code: if works == false, will throw an error    Scala 代码，如果 works == false，那么将会报错
println("成功！！") // Scala Code: if we get here, our tests passed! 如果我们运行到了这里，那么我们的验证就通过啦！

<span style="color:blue">**Example: Optional IO with Zero-Width Wires | 举例：可选择的带零位宽连线的IO**</span><br>
Another way to achieve similar functionality to `Option`s is with zero-width wires. 
Chisel types are allowed to have widths of zero. 
An IO with width zero is pruned from the emitted Verilog, and anything that tries to use the value of a zero-width wire gets a constant zero. 
If zero is a sensible default value, zero-width wires can be nice because they obviate the need for matching on an option or calling `getOrElse`. 

另外一个实现相似功能的 `Option` 是使用零位宽连线。
Chisel 的类型允许零位宽。
一个零位宽的 IO 在生成 Verilog 时会被剪切掉，所有尝试使用零位宽的连线都会接低电平。
如果零是一个默认的敏感信号，那么零位宽连线将会是一个非常棒的选择，因为它避免了匹配的需求或者称为 `getOrElse` 。

In [ ]:
class HalfFullAdder(val hasCarry: Boolean) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val carryIn = Input(if (hasCarry) UInt(1.W) else UInt(0.W))
    val s = Output(UInt(1.W))
    val carryOut = Output(UInt(1.W))
  })
  val sum = io.a +& io.b +& io.carryIn
  io.s := sum(0)
  io.carryOut := sum(1)
}
println("Half Adder:")
println(getVerilog(new HalfFullAdder(false)))
println("\n\nFull Adder:")
println(getVerilog(new HalfFullAdder(true)))

---
# Implicits | 隐式方法
There are often times when you are programming that requires a lot of boilerplate code. To handle this use case, Scala introduced the notion of **implicits**, which allow the compiler to do some syntactic sugar for you. Because lots of things happen behind the scenes, implicits can appear very magical. This section breaks down some basic examples to explain what they are and where they are commonly used. 

有时候你的设计需要很多的样板代码。为了在这些情况下更加方便， Scala 引入了**隐式方法**的概念，这使得编译器可以为你在句法方面有不少帮助。因为在背后，隐式方法会展现它的魔力。这个小节把一些基础的例子拆散，向你解释都有哪些隐式方法、在什么样的情况下会使用。

## Implicit Arguments | 隐式参数
At times, your code will require accessing a top-level variable of some sort from deep within a series of function calls. Instead of manually threading this variable through every function call, you can use implicit arguments to do it for you. 

有很多时候，你的代码需要从顶层获取一个变量，而不想要好几次引用函数。你可以使用隐式参数而不需要手动地穿过每一层函数，一连串地引用这个变量。

<span style="color:blue">**Example: Implicit Cats | 举例：隐式猫**</span><br>
In the following example, we can pass the number of cats implicitly or explicitly. 

在这个例子中，我们能够隐式或者显式地传递`猫`的数量。

In [ ]:
object CatDog {
  implicit val numberOfCats: Int = 3
  //implicit val numberOfDogs: Int = 5

  def tooManyCats(nDogs: Int)(implicit nCats: Int): Boolean = nCats > nDogs
    
  val imp = tooManyCats(2)    // Argument passed implicitly! 隐式参数传递！
  val exp = tooManyCats(2)(1) // Argument passed explicitly! 显示参数传递！
}
CatDog.imp
CatDog.exp

What's happening here? First, we define an implicit value **numberOfCats**. In a given scope, **there can only be one implicit value of a given type**. Then, we define a function that takes two argument lists; the first is any explicit parameters, and the second are any implicit parameters. When we call **tooManyCats**, we either omit the second implicit argument list (letting the compiler find it for us), or explicitly provide an argument (which can be different than the implicit value). 

The following are ways implicit arguments can *fail*: 
- Two or more implicit values of a given type are defined in a scope 
- If the compiler cannot find an implicit value necessary for a function call 

在这背后发生了什么呢？首先，我们定义了一个隐式值 **numberOfCats**。在一个给定的情况下，**只能有一个含有一个给定类型的隐式值**。然后，我们定义了一个有两个参数列表的函数式，第一个参数列表是所有的显式参数，第二个参数列表是所有的隐式参数。当我们调用函数 **tooManyCats** 时，我们要么省略第二个隐参数列表（让编译器为我们找到它），或者明确的提供一个参数（与隐式值不同）。

下面这些隐式参数的方法会*失败*：
- 在同一个给定的情况中，同一个给定的类型下有两个或者更多隐式值
- 如果编译器不能在调用函数值的时候找到一个必要的隐式值

<span style="color:blue">**Example: Implicit Logging | 举例：隐式日志**</span><br>
The next code block shows how you might use implicit arguments to implement logging in a Chisel generator. 

***Note: there are better ways to do logging in Scala! ***

下面的这个代码块展示了在 Chisel 生成器中，你可以怎样使用隐式参数来记录日志

***注意：在 Scala 中有更棒的方法记录日志***

In [ ]:
sealed trait Verbosity
implicit case object Silent extends Verbosity
case object Verbose extends Verbosity

class ParameterizedWidthAdder(in0Width: Int, in1Width: Int, sumWidth: Int)(implicit verbosity: Verbosity)
extends Module {
  def log(msg: => String): Unit = verbosity match {
    case Silent =>
    case Verbose => println(msg)
  }
  require(in0Width >= 0)
  log(s"in0Width of $in0Width OK")
  require(in1Width >= 0)
  log(s"in1Width of $in1Width OK")
  require(sumWidth >= 0)
  log(s"sumWidth of $sumWidth OK")
  val io = IO(new Bundle {
    val in0 = Input(UInt(in0Width.W))
    val in1 = Input(UInt(in1Width.W))
    val sum = Output(UInt(sumWidth.W))
  })
  log("Made IO")
  io.sum := io.in0 + io.in1
  log("Assigned output")
}

println(getVerilog(new ParameterizedWidthAdder(1, 4, 5)))
println(getVerilog(new ParameterizedWidthAdder(1, 4, 5)(Verbose)))

## Implicit Conversions | 隐式转换
Like implicit arguments, implicit functions (also known as **implicit conversions**) are used to reduce boilerplate code. More specifically, they are used to automatically convert one Scala object into another. 

就像隐式参数，隐式函数（或者也叫**隐式转换**）经常被用于减少模板代码。更具体地说，他们通常用于自动将一种 Scala 对象转化为另外一种。

<span style="color:blue">**Example: Implicit Conversion | 举例：隐式转换**</span><br>
In the following example, we have two classes, `Animal` and `Human`. `Animal` has a `species` field, but `Human` does not. However, by implementing an implicit conversion, we can call `species` on a `Human`. 

在下面这个例子中，我们有两个类，`Animal` 和 `Human`，类`Animal` 有一个变量 `species`，但是类 `Human` 没有。然而，通过隐式转换的实现，我们可以在类 `Human` 中调用 `species`

In [ ]:
class Animal(val name: String, val species: String)
class Human(val name: String)
implicit def human2animal(h: Human): Animal = new Animal(h.name, "Homo sapiens")
val me = new Human("Adam")
println(me.species)

Generally, implicits can make your code confusing, so we recommend you use them as a last resort. First try inheritance, traits, or method overloading.

通常，隐式方法可以让你的代码变得可配置，所以我们建议你把隐式方法当作最后一种手段。首先尝试继承、特质或者方法重载。

---
# Generator Example | 生成器举例
The following example shows a generator for a 1-bit input Mealy machine. 
Read through the code and try to follow what's going on. 

以下的例子展示了一个一位输入的 Mealy 状态机
在[维基百科](https://en.wikipedia.org/wiki/Mealy_machine#/media/File:Mealy.png)有一个验证的例子。
阅读下面的代码并且试图弄清楚每一步在干什么。


<span style="color:blue">**Example: Mealy Machine | 举例：Mealy 状态机**</span><br>
Try making your own parameterizations of the Mealy machine generator and writing your own tests in the code block below. 

尝试在这个 Mealy 状态机生成器中使用你自己的参数，并且在后面的代码块中完成你自己的验证。

In [ ]:
// Mealy机
case class BinaryMealyParams(
  // number of states 状态的数量
  nStates: Int,
  // initial state 初始状态
  s0: Int,
  // function describing state transition 状态转移函数
  stateTransition: (Int, Boolean) => Int,
  // function describing output 输出函数
  output: (Int, Boolean) => Int
) {
  require(nStates >= 0)
  require(s0 < nStates && s0 >= 0)
}

class BinaryMealy(val mp: BinaryMealyParams) extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val out = Output(UInt())
  })

  val state = RegInit(UInt(), mp.s0.U)

  // output zero if no states
  io.out := 0.U
  for (i <- 0 until mp.nStates) {
    when (state === i.U) {
      when (io.in) {
        state  := mp.stateTransition(i, true).U
        io.out := mp.output(i, true).U
      }.otherwise {
        state  := mp.stateTransition(i, false).U
        io.out := mp.output(i, false).U
      }
    }
  }
}

// 来自 https://en.wikipedia.org/wiki/Mealy_machine 的例子
val nStates = 3
val s0 = 2
def stateTransition(state: Int, in: Boolean): Int = {
  if (in) {
    1
  } else {
    0
  }
}
def output(state: Int, in: Boolean): Int = {
  if (state == 2) {
    return 0
  }
  if ((state == 1 && !in) || (state == 0 && in)) {
    return 1
  } else {
    return 0
  }
}

val testParams = BinaryMealyParams(nStates, s0, stateTransition, output)

class BinaryMealyTester(c: BinaryMealy) extends PeekPokeTester(c) {
  poke(c.io.in, false)
  expect(c.io.out, 0)
  step(1)
  poke(c.io.in, false)
  expect(c.io.out, 0)
  step(1)
  poke(c.io.in, false)
  expect(c.io.out, 0)
  step(1)
  poke(c.io.in, true)
  expect(c.io.out, 1)
  step(1)
  poke(c.io.in, true)
  expect(c.io.out, 0)
  step(1)
  poke(c.io.in, false)
  expect(c.io.out, 1)
  step(1)
  poke(c.io.in, true)
  expect(c.io.out, 1)
  step(1)
  poke(c.io.in, false)
  expect(c.io.out, 1)
  step(1)
  poke(c.io.in, true)
  expect(c.io.out, 1)
}
val works = iotesters.Driver(() => new BinaryMealy(testParams)) { c => new BinaryMealyTester(c) }
assert(works) // Scala Code: if works == false, will throw an error Scala代码，如果 works == false，那么将会报错
println("成功！！") // Scala Code: if we get here, our tests passed! Scala代码，如果我们运行到了这里，那我们的验证就通过了！

---
# You're done! | 恭喜你，完成了本节内容的学习！

[Return to the top. | 回到顶层](#top)